# Keras API

## Sequential model 



In [ ]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential([Dense(2, input_dim = 1), Dense(1)])  # inside are layers

In [ ]:
# alternatively, add layers piecewise

model = Sequential()
model.add(Dense(2, input_dim = 1))
model.add(Dense(1))

## Functional models 

More flexible than sequential, allows multiple input and output models, and shared layer models. 


In [1]:
# Standard imports: 
from keras.utils import plot_model
from keras.layers import Input
from keras.layers import Dense
from keras.models import Model

### 1. inputs

(Compared to Sequential) Must create and define a standalone input layer that specifies the shape of input data. `shape` arguments need a tuple. Input is a tensor. Shape seems to be the column number of the input. 

When input is 1-d, shape must leave room for the shape of the mini-batch size used when splitting the data when training the network. The shape tuple is always defined with a hanging last dimension. 

In [ ]:
from keras.layers import Input
visible = Input(shape = (2,))

### 2. connecting layers

Layers are connected pairwise, by specifying where the inputs come from when defining each new layer. Use bracket notation.


In [ ]:
from keras.layers import Dense
hidden = Dense(2)(visible)  # receives input only from input layer

### 3. create a model 

Only need to specify the input and output layers. 


In [ ]:
from keras.models import Model
model = Model(inputs = visible, outputs = hidden)

# Standard network models 

## 1. Multilayer perceptron

- Input: 10
- Hidden layers: 3
    - 10 neurons
    - 20 neurons
    - 10 neurons
- Output layer: 1 output 
- activation function: rectified linear (relu) for hidden layer, sigmoid activation for output (binary classificatio)

Number of parameters: use number of neurons in the left-hand-side times right-hand-side, then add right-hand-side as bias term. 

E.g. 

- dense_1: 10 (input) $\times$ 10 (layer1) + 10(layer1 bias) $= 110$
- dense_2: 10 (layer1) $\times$ 20 (layer2) + 20(layer2 bias) $= 220$
- dense_3: 20 (layer2) $\times$ 10 (layer3) + 10(layer3 bias) $= 210$
- output: 10 (layer3) $\times$ 1 (output) + 1(output bias) $= 11$

Total number of parameter is 551.

In [2]:
visible = Input(shape=(10,))
hidden1 = Dense(10, activation='relu')(visible)
hidden2 = Dense(20, activation='relu')(hidden1)
hidden3 = Dense(10, activation='relu')(hidden2)
output = Dense(1, activation='sigmoid')(hidden3)
model = Model(inputs=visible, outputs=output)
# summarize layers
print(model.summary())
# plot graph
# plot_model(model, to_file='multilayer_perceptron_graph.png')


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 20)                220       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 551
Trainable params: 551
Non-trainable params: 0
_________________________________________________________________
None


## 2. Convolutional NN

## 3. Recurrent NN (LSTM)

- Input: 100 time steps of one feature
- Hidden: 
    - 1 single LSTM to extract features
    - 1 fully connected to interpret LSTM output
- Output: binary predictions 

In [3]:
# in addition to those imported, 

from keras.layers.recurrent import LSTM  # is it different if omit .recurrent? 

In [4]:
visible = Input(shape=(100,1))
hidden1 = LSTM(10)(visible)
hidden2 = Dense(10, activation='relu')(hidden1)
output = Dense(1, activation='sigmoid')(hidden2)   # classification
model = Model(inputs=visible, outputs=output)
# summarize layers
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100, 1)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                480       
_________________________________________________________________
dense_5 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 601
Trainable params: 601
Non-trainable params: 0
_________________________________________________________________
None


## Tornament data example (from DC)

Took me a while to get a subset of the data. They should have put it downloadable... 

Goal: predict tornament outcomes 

Input: seed difference. A 7 (seed) team vs 10 team is -3. 

- seed ranges from 1 to 16, so seed difference is between 15 to -15.
- positive seed: negative outcome. 

Output: score difference. 41 - 50 is -9. (it means the first team loses by 9 points)

In [22]:
import pandas as pd
import numpy as np

In [26]:
tornament = pd.read_csv('/Users/andrea/Documents/Data/pyDataCamp/game_tornament.csv', sep = ';')
tornament.head()

,season,team_1,team_2,home,seed_diff,score_diff,score_1,score_2,won
0,1985,288,73,0,-3,-9,41,50,0
1,1985,5929,73,0,4,6,61,55,1
2,1985,9884,73,0,5,-4,59,63,0
3,1985,73,288,0,3,9,50,41,1
4,1985,3920,410,0,1,-9,54,63,0


In [29]:
tornament.columns.values   # note the bug of the blank spaces

array(['season', 'team_1', '  team_2', '  home  ', 'seed_diff ',
       ' score_diff ', ' score_1 ', ' score_2 ', ' won'], dtype=object)

In [30]:
# this is after you have compiled model 

model.fit(tornament['seed_diff '], tornament[' score_diff '],
          epochs=1,
          batch_size=128,
          validation_split=0.1,
          verbose=True)

Instructions for updating:
Use tf.cast instead.
Train on 325 samples, validate on 37 samples
Epoch 1/1
325/325 [==============================] - 0s 698us/step - loss: 13.9641 - val_loss: 12.5979


In [31]:
'''
# Load the X variable from the test data
X_test = games_tourney_test['seed_diff']

# Load the y variable from the test data
y_test = games_tourney_test['score_diff']

# Evaluate the model on the test data
model.evaluate(X_test, y_test)
'''

"\n# Load the X variable from the test data\nX_test = games_tourney_test['seed_diff']\n\n# Load the y variable from the test data\ny_test = games_tourney_test['score_diff']\n\n# Evaluate the model on the test data\nmodel.evaluate(X_test, y_test)\n"